In [1]:
import pandas as pd 
from db import *
from config_poc import *

```
select  c.table_name, c.column_name, c.data_type, c.*
from information_schema.columns c
where 1=1
and c.table_catalog = 'faculty-Cornell-CS'
and c.table_schema = 'main'
and c.table_name in (
	--'t_note'
	select table_name from information_schema.tables t
	where t.table_catalog = 'faculty-Cornell-CS'
	and t.table_schema = 'main'
)
order by c.table_catalog,c.table_schema,c.table_name, c.ordinal_position
```

In [2]:
sql_stmt = """
select  c.table_name, c.column_name --, c.data_type,  c.*
from information_schema.columns c
where 1=1
--and c.table_catalog like 'faculty-Cornell-CS%'
and c.table_schema = 'main'
and c.table_name in (
	--'t_note'
	select table_name from information_schema.tables t
	where 1=1
	--and t.table_catalog like 'faculty-Cornell-CS%'
	and t.table_schema = 'main'
)
and c.table_name not like 't1_%'
order by c.table_catalog,c.table_schema,c.table_name, c.ordinal_position
;
"""

In [3]:
with DBConn() as _conn:
    df = pd.read_sql(sql_stmt, _conn)

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_12452\496018617.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_stmt, _conn)


In [4]:
df

,table_name,column_name
0,t_column_props,table_name
1,t_column_props,col_name
2,t_column_props,is_system_col
3,t_column_props,is_user_key
4,t_column_props,is_required
...,...,...
68,t_work,tags
69,t_work,note
70,t_work,ts_created
71,t_work,id


In [18]:
df.to_csv("tab_cols.csv", index=False)

In [5]:
df_list= df.groupby('table_name')['column_name'].apply(list)

In [6]:
TABLE_COLUMNS = df_list.to_dict()

In [7]:
TABLE_COLUMNS

{'t_column_props': ['table_name',
  'col_name',
  'is_system_col',
  'is_user_key',
  'is_required',
  'is_visible',
  'is_editable',
  'is_clickable',
  'form_column',
  'widget_type',
  'label_text'],
 't_faculty': ['name',
  'job_title',
  'phd_univ',
  'phd_year',
  'research_area',
  'research_concentration',
  'research_focus',
  'url',
  'img_url',
  'phone',
  'email',
  'cell_phone',
  'office_address',
  'department',
  'school',
  'note'],
 't_note': ['title', 'note', 'url', 'tags', 'ts', 'id', 'ref_type', 'ref_key'],
 't_org': ['name', 'url', 'note'],
 't_person': ['name',
  'url',
  'email',
  'first_name',
  'mid_name',
  'last_name',
  'note'],
 't_person_team': ['id',
  'ts',
  'ref_type',
  'ref_key',
  'ref_type_2',
  'ref_key_2'],
 't_person_work': ['id',
  'ts',
  'ref_type',
  'ref_key',
  'ref_type_2',
  'ref_key_2'],
 't_research_group': ['research_group', 'url', 'note'],
 't_team': ['name', 'url', 'note'],
 't_work': ['type',
  'title',
  'url',
  'summary',
  '

In [8]:
PROPS

['is_system_col',
 'is_user_key',
 'is_required',
 'is_visible',
 'is_editable',
 'is_clickable',
 'form_column',
 'widget_type',
 'label_text']

In [9]:
TABLE_COLUMNS.keys()

dict_keys(['t_column_props', 't_faculty', 't_note', 't_org', 't_person', 't_person_team', 't_person_work', 't_research_group', 't_team', 't_work'])

In [10]:
COLUMN_PROPS.keys()

dict_keys(['t_note', 't_team', 't_work', 't_person_team', 't_person_work'])

In [11]:
selected_tbls = [t for t in TABLE_COLUMNS.keys() if t not in [
            "t_person_team", "t_person_work", "t_team", "t_work", 't_note', 't_column_props']]

In [12]:
selected_tbls

['t_faculty', 't_org', 't_person', 't_research_group']

In [13]:
def _gen_label(col):
    "Convert table column into form label"
    if col == 'ts_created': return "Created At"
    if "_" not in col:
        if col.upper() in ["URL","ID"]:
            return col.upper()
        elif col.upper() == "TS":
            return "Timestamp"
        return col.capitalize()

    cols = []
    for c in col.split("_"):
        c  = c.strip()
        if not c: continue
        cols.append(c.capitalize())
    return " ".join(cols)

In [14]:
def _get_columns(table_name, prop_name="is_visible"):
    cols_bool = []
    cols_text = {}
    for k,v in COLUMN_PROPS[table_name].items():
        if prop_name.startswith("is_") and v.get(prop_name, False):
            cols_bool.append(k)
            
        if not prop_name.startswith("is_"):
            val = v.get(prop_name, "")
            if val:
                cols_text.update({k: val})
    
    return cols_bool or cols_text

In [15]:
def _parse_column_props():
    """parse COLUMN_PROPS map
    """
    col_defs = {}
    for table_name in COLUMN_PROPS.keys():
        defs = {}
        cols_widget_type = {}
        cols_label_text = {}
        for p in PROPS:
            res = _get_columns(table_name, prop_name=p)
            # print(f"{p}: {res}")
            if p == 'widget_type':
                cols_widget_type = res
            elif p == 'label_text':
                cols_label_text = res
            defs[p] = res
            
        # reset label
        for col in cols_widget_type.keys():
            label = cols_label_text.get(col, "")
            if not label:
                label = _gen_label(col)
            cols_label_text.update({col : label})
        # print(cols_label_text)
        defs['label_text'] = cols_label_text
        col_defs[table_name] = defs
        
    return col_defs

In [16]:
_parse_column_props()

{'t_note': {'is_system_col': ['id', 'ts'],
  'is_user_key': ['name'],
  'is_required': ['name', 'id'],
  'is_visible': ['name', 'url', 'tags', 'id', 'note'],
  'is_editable': ['name', 'url', 'tags', 'note'],
  'is_clickable': ['url'],
  'form_column': {'name': 'col1-1',
   'url': 'col1-2',
   'tags': 'col1-3',
   'id': 'col2-1',
   'note': 'col2-2'},
  'widget_type': {'name': 'text_input',
   'url': 'text_input',
   'tags': 'text_input',
   'id': 'text_input',
   'note': 'text_area'},
  'label_text': {'url': 'URL',
   'id': 'ID',
   'name': 'Name',
   'tags': 'Tags',
   'note': 'Note'}},
 't_team': {'is_system_col': ['id'],
  'is_user_key': ['name'],
  'is_required': ['name', 'id'],
  'is_visible': ['name', 'url', 'id', 'note'],
  'is_editable': ['name', 'url', 'note'],
  'is_clickable': ['url'],
  'form_column': {'name': 'col1-1',
   'url': 'col1-2',
   'id': 'col2-1',
   'note': 'col2-2'},
  'widget_type': {'name': 'text_input',
   'url': 'text_input',
   'id': 'text_input',
   'note

In [1]:
s = "publication, preprint, talk, poster, project, startup, company"
WORK_TYPES = [c.strip() for c in s.split(",") if c.strip()]
WORK_TYPES

['publication', 'preprint', 'talk', 'poster', 'project', 'startup', 'company']

In [19]:
form_cols = {
    "note": "col2-2",
    "url": "col1-2",
    "name": "col1-1",
    "id": "col2-1",
}

res = {}
for i in range(1,4):
    m = {k:v for k,v in form_cols.items() if v.startswith(f"col{i}-")}
    res[f"col{str(i)}_columns"] = sorted(m, key=m.__getitem__)

In [20]:
res

{'col1_columns': ['name', 'url'],
 'col2_columns': ['id', 'note'],
 'col3_columns': []}

In [21]:
res2 = {}

In [22]:
res2.update(res)

In [23]:
res2

{'col1_columns': ['name', 'url'],
 'col2_columns': ['id', 'note'],
 'col3_columns': []}